### 1. 为什么使用RabbitMQ？

>**什么是RMQ**：  
    采用AMQP高级消息队列协议的一种消息队列技术,最大的特点就是消费并不需要确保提供方存在,实现了服务之间的高度解耦。

**削峰：**减少高峰时期对数据库的压力， 防止所有的请求一次性直接打到数据库中，导致数据库崩溃。

当然，RMQ还有其他的优点：
1. **解耦**：一个系统或者一个模块，调用了多个系统或者模块，互相之间的调用很复杂，维护起来很麻烦。但是其实这个调用是不需要直接同步调用接口的，如果用 MQ 给它异步化解耦
2. **异步：** 生产者生产后，放入队列，不用等消费者消费之后再返回，可以通过轮寻的方式定期查看消费结果；异步发送短信、推送消息、⽇志记录等，可以⼤⼤减⼩响应时间
3. **可靠性:** RabbitMQ使用一些机制来保证可靠性， 如持久化、传输确认及发布确认等。

# 下面的2，3，4均是保证**可靠性**的方法

### 2. 如何确保消息正确地发送至RabbitMQ？如何确保消息接收方消费了消息？

**发送方确认模式**

1. 将信道设置成**confirm模式（发送方确认模式）**，则所有在信道上发布的消息都会被指派一个唯一的ID。
2. 一旦消息被**投递**到目的队列后，或者消息被**写入磁盘**后（可持久化的消息），信道会发送一个确认给生产者（包含消息唯一 ID）。
3. 如果 RabbitMQ发生内部错误从而导致消息丢失，会发送一条**nack（notacknowledged，未确认）**消息。发送方确认模式是异步的，生产者应用程序在等待确认的同时，可以继续发送消息。当确认消息到达生产者应用程序，生产者应用程序的**回调**方法就会被触发来处理确认消息。

**接收方确认机制**  
    **接收方消息确认机制**  
    
* 消费者接收每一条消息后都必须进行**确认**（消息接收和消息确认是两个不同操作）。
* 只有消费者确认了消息，RabbitMQ才能安全地把消息从队列中删除。
* 这里并没有用到超时机制，RabbitMQ仅通过Consumer的连接中断来确认是否需要重新发送消息。也就是说，只要连接不中断，RabbitMQ给了Consumer足够长的时间来处理消息。保证数据的最终一致性；
* **如果消费者接收到消息，在确认之前断开了连接或取消订阅，RabbitMQ会认为消息没有被分发，然后重新分发给下一个订阅的消费者**
* 如果消费者接收到消息却没有确认消息，连接也未断开，则RabbitMQ认为该消费者繁忙，将不会给该消费者分发更多的消息。


### 3. 如何避免消息重复投递或重复消费？

**为什么会重复消费**  

正常情况下，消费者在消费消息的时候，消费完毕后，会发送一个确认消息给消息队列，消息队列就知道该消息被消费了，就会将该消息从消息队列中删除；

但是因为网络传输等等故障，确认信息没有传送到消息队列，导致消息队列不知道自己已经消费过该消息了，再次将消息分发给其他的消费者。


**避免重复消费**

* 1. 在传递的消息中设置一个唯一的标识（如支付ID、订单ID、帖子ID等）
* 2. 使用外部介质 + 设置一个唯一标识；如redis；每次在消费前先查询是否被消费，如果没有，则消费且设置value为已消费

**避免重复投递**

* 在消息生产时，MQ内部针对每条生产者发送的消息生成一个inner-msg-id

### 4. 如何解决丢数据的问题?

* 丢失又分为：生产者丢失消息、消息列表丢失消息、消费者丢失消息；  


1. **生产者丢失消息：**   

从生产者弄丢数据这个角度来看，RabbitMQ提供**transaction(事务)和confirm**模式来确保生产者不丢消息；
* **transaction机制**就是说：发送消息前，开启事务（channel.txSelect()）,然后发送消息，如果发送过程中出现什么异常，事务就会回滚（channel.txRollback()）,如果发送成功则提交事务（channel.txCommit()）。**缺点：吞吐量下降；**  


* **confirm模式**（用的居多）：一旦channel进入confirm模式，所有在该信道上发布的消息都将会被指派一个唯一的ID（从1开始），一旦消息被投递到所有匹配的队列之后；rabbitMQ就会发送一个**ACK**给生产者（包含消息的唯一ID），这就使得生产者知道消息已经正确到达目的队列了；**如果**rabbitMQ没能处理该消息，则会发送一个Nack消息给你，你可以进行重试操作。


2. **消息队列丢数据：消息持久化，开启将消息持久化写入磁盘**


* 这个持久化配置可以和confirm机制配合使用，你可以在消息持久化磁盘后，再给生产者发送一个Ack信号。这样，如果消息持久化磁盘之前，rabbitMQ阵亡了，那么生产者收不到Ack信号，生产者会自动重发。  

**开启持久化(消息持久化)**
    1. 将queue的持久化标识durable设置为true,则代表是一个持久的队列
    2. 发送消息的时候将deliveryMode=2
这样设置以后，即使rabbitMQ挂了，重启后也能恢复数据


3. **消费者丢失消息：**
取消自动确认，采用手动确认；处理消息成功后，手动回复确认消息。

### 5. 持久化的三种类型

1. 交换机（Exchange)持久化
```java
/**
         * 参数1：交换机名称
         * 参数2：交换机类型
         * 参数3：是否持久化 默认 false
         */
        channel.exchangeDeclare("logs_direct", BuiltinExchangeType.DIRECT,true);
```

2. 队列（Queue）持久化  

```java
/**
         * 参数1：String queue 队列名称 如果队列不存在会自动创建
         * 参数2：boolean durable  队列是否持久化 true 持久化 false 不持久化  默认：false
         * 参数3：boolean exclusive 是否独占队列 true 独占队列 false 不独占 默认：true
         * 参数4：boolean autoDelete 是否在消费完成后自动删除  true 自动删除 默认：true
         * 参数5：Map<String, Object> arguments  额外附加参数
         */
        channel.queueDeclare("hello-1",true,false,false,null);

```

3. 消息（Message）持久化  

```java
channel.basicPublish("exchangeName" , "routingKey",
                new AMQP.BasicProperties.Builder()
                        .deliveryMode(2)
                        .build(),
                "ddf".getBytes());

```

### 6. 4种交换机模式（RabbitMq是怎么路由的）

* **Direct Exchange**：此交换机需要绑定一个队列，要求该消息与一个特定的路由键完全匹配。简单点说就是一对一的，点对点的发送。
* 发送的时候需要 指定交换机，发送的key，这样交换机会将消息发送给 绑定了这个key的队列

* 定义队列，定义交换机
* 让队列绑定一个key
* 让交换机绑定队列和相应的key

* **Fanout Exchange**：类似**广播**模式；将队列绑定到交换机上。一个发送到交换机的消息都会被转发到与该交换机绑定的所有队列上

* 定义队列，定义交换机
* 让队列绑定交换机
* 这样客户端 通过指定消息要发送的交换机，交换机会直接把消息发送给所有绑定了该交换机的队列

* **Topic Exchange**：主题交换机，使用通配符去匹配，路由到对应的队列。


* 通配符有两种："\*" 、 "\#"。需要注意的是通配符前面必须要加上"."符号。
* \* 符号：有且只匹配一个词。比如 a.*可以匹配到"a.b"、"a.c"，但是匹配不了"a.b.c"。
* \# 符号：匹配一个或多个词。比如"rabbit.#"既可以匹配到"rabbit.a.b"、"rabbit.a"，也可以匹配到"rabbit.a.b.c"。


* 定义队列，定义交换机
* 定义交换机要绑定的key和队列，
* 定义队列需要匹配的key
* 当客户端发送消息给交换机时，交换机给根据key的匹配原则，将消息发送给匹配到的队列

* **Headers Exchange**：路由不是用routingKey进行路由匹配，而是在匹配请求头中所带的键值进行路由;;创建队列需要设置绑定的头部信息，有两种模式：全部匹配和部分匹配。交换机会根据生产者发送过来的头部信息携带的键值去匹配队列绑定的键值，路由到对应的队列。

### 7. MQ的组成

![](img/mq.png)

* Broker：可以看做RabbitMQ的服务节点。一般情况下，一个broker可以看做一个RabbitMQ服务器
* Exchange：消息交换机，它指定消息按什么规则，路由到哪个队列。
* Queue：消息队列载体，每个消息都会被投⼊到⼀个或多个队列。
* Binding：绑定，它的作⽤就是把exchange和queue按照路由规则绑定起来。
* Routing Key：路由关键字，exchange根据这个关键字进⾏消息投递。
* vhost：虚拟主机，⼀个broker⾥可以开设多个vhost，⽤作不同⽤户的权限分离；每个VirtualHost相当于一个相对独立的RabbitMQ服务器；每个VirtualHost之间是相互隔离的，exchange、queue、message不能互通。 

* producer：消息⽣产者，就是投递消息的程序。
* consumer：消息消费者，就是接受消息的程序。
* channel：消息通道，在客户端的每个连接⾥，可建⽴多个channel，每个channel代表⼀个会话任务。

**channel详情**：信道是建立在 Connection 之上的虚拟连接，RabbitMQ 处理的每条 AMQP 指令都是通过信道完成的。
* **RabbitMQ 使用信道的方式来传输数据**。信道是建立在真实的 TCP 连接内的虚拟连接，且每条 TCP 连接上的信道数量没有限制。

**采用信道(channel)而不是直接用TCP的原因：** 对于操作系统而言，建立和销毁 TCP 连接是非常昂贵的开销，如果遇到使用高峰，性能瓶颈也随之显现。

RabbitMQ 选择 TCP 连接复用，不仅可以减少性能开销，同时也便于管理。

![](img/channel.png)

### 8. AMQP协议

AMQP（Advanced Message Queuing Protocol，高级消息队列协议）是一个**进程间传递异步消息的网络协议。**

工作原理：发布者（Publisher）发布消息（Message），经由交换机（Exchange）。

交换机根据**路由规则**将收到的消息分发给与该交换机绑定的队列（Queue）。

最后 AMQP 代理会将消息投递给订阅了此队列的消费者，或者消费者按照需求自行获取。

交换机（Exchange），队列（Queue），绑定（binding）是AMQP模型的三大组件

AMQP协议3层:  
* **Module Layer:** 协议最高层，主要定义了一些客户端调用的命令，客户端可以用这些命令实现自己的业务逻辑。
* **Session Layer:** 中间层，主要负责客户端命令发送给服务器，再将服务端应答返回客户端，提供可靠性同步机制和错误处理。
* **TransportLayer:** 最底层，主要传输二进制数据流，提供帧的处理、信道服用、错误检测和数据表示等


### 9. 死信队列

* 死信：消息的一种状态，它仍然是一个正常的消息；
    
    从消息变成死信：
    
        1、消息过期。消息在队列的存活时间超过所设置的 TTL 时间。
        2、消息被拒绝，调用了 channel.basicNack 或 channel.basicReject方法，并且设置 requeue 参数为false。
        3、队列的消息达到最大长度。


DLX ，全称为 Dead-Letter-Exchange ，可以称之为死信交换机。当消息在一个队列中变成死信（ dead message ）之后，它能被重新被发送到另一个交换器中，这个交换器就是 DLX ，绑定 DLX 的队列就称之为死信队列。

DLX 也是一个**正常的交换器**，和一般的交换器没有区别，它能在任何的队列上被指定，实际上就是设置某个队列的属性。当这个队列中存在死信时 RabbitMQ 就会自动地将这个消息新发布到设置的 DLX 上去，进而被路由到另一个队列，即死信队列。可以监听这个队列中的消息、以进行相应的处理。

### 11. 集群  

1. 通常情况下，在集群中我们把**每一个服务称之为一个节点**，在 RabbitMQ 集群中，节点类型可以分为两种：

2. **内存节点**：元数据存放于内存中。为了重启后能同步数据，内存节点会将磁盘节点的地址存放于磁盘之中，除此之外，如果消息被持久化了也会存放于磁盘之中，因为内存节点读写速度快，一般客户端会连接内存节点。

3. **磁盘节点**：元数据存放于磁盘中（默认节点类型），需要保证至少一个磁盘节点，否则一旦宕机，无法恢复数据，从而也就无法达到集群的高可用目的。

* 元数据，指的是包括队列名字属性、交换机的类型名字属性、绑定信息、vhost等基础信息，不包括队列中的消息数据

**普通集群**：将 RabbitMQ 部署到多台服务器上，每个服务器启动一个 RabbitMQ 实例，多个实例之间进行消息通信。集群中各个节点之间**只会相互同步元数据**，也就是说，消息数据不会被同步；

* 当我们消费消息的时候，如果连接到了另外一个实例，那么那个实例会通过元数据定位到 Queue 所在的位置，然后访问 Queue 所在的实例，拉取数据过来发送给消费者。
* 缺点：无法保证高可用，如果保存数据的结点宕机了，可能会导致数据丢失

![](img/cluster01.png)

**镜像集群**：节点之间**不仅仅会同步元数据，消息内容也会在镜像节点间同步**，可用性更高，但是可能会有一些延迟

![](img/cluster02.png)

### 12. 上千万条消息在mq中积压了几个小时还没解决：

1. 先修复consumer的问题，确保其恢复消费速度，然后将现有consumer都停掉；
2. 新建⼀个topic，partition是原来的10倍，临时建⽴好原先10倍或者20倍的queue数量；
3. 然后写⼀个临时的分发数据的consumer程序，这个程序部署上去消费积压的数据；消费之后不做耗时的处理，直接均匀轮询写⼊临时建⽴好的10倍数量的queue；
4. 接着临时征⽤10倍的机器来部署consumer，每⼀批consumer消费⼀个临时queue的数据；
5. 这种做法相当于是临时将queue资源和consumer资源扩⼤10倍，以正常的10倍速度来消费数据；
6. 等快速消费完积压数据之后，得恢复原先部署架构，重新⽤原先的consumer机器来消费消息。

### 13. 消息如何分发？
若该队列⾄少有⼀个消费者订阅，消息将以**循环（round-robin）**的⽅式发送给消费者。每条消息只会分发给⼀个订阅的消费者（前提是消费者能够正常处理消息并进⾏确认）

### 14. 如何保证消息的顺序性？
通过某种算法，将需要保持先后顺序的消息放到同⼀个消息队列中(kafka中就是partition,rabbitMq中就是queue)。然后只⽤⼀个消费者去消费该队列。
可以在消息体内添加全局有序标识来实现。

### 15. RabbitMQ中消息可能有的几种状态?
* alpha: 消息内容(包括消息体、属性和 headers) 和消息索引都存储在内存中 。

* beta: 消息内容保存在磁盘中，消息索引保存在内存中。

* gamma: 消息内容保存在磁盘中，消息索引在磁盘和内存中都有 。

* delta: 消息内容和索引都在磁盘中 。

### 16. 延时队列

在rabbitmq中不存在延时队列，但是我们可以**通过设置消息的过期时间和死信队列来模拟出延时队列**。消费者监听死信交换器绑定的队列，而不要监听消息发送的队列

### 17. 队列结构

* rabbit_amqqueue_process:负责协议相关的消息处理，即接收生产者发布的消息、向消费者交付消息、处理消息的确认(包括生产端的 confirm 和消费端的 ack) 等。

* backing_queue:是消息存储的具体形式和引擎，并向 rabbit amqqueue process提供相关的接口以供调用。

### 18. 优先级队列？
* 优先级高的队列会先被消费。

* 可以通过x-max-priority参数来实现。

* 当消费速度大于生产速度且Broker没有堆积的情况下，优先级显得没有意义。

* **生产者消息运转？**  

1.Producer先连接到Broker,建立连接Connection,开启一个信道(Channel)。

2.Producer声明一个交换器并设置好相关属性。

3.Producer声明一个队列并设置好相关属性。

4.Producer通过路由键将交换器和队列绑定起来。

5.Producer发送消息到Broker,其中包含路由键、交换器等信息。

6.相应的交换器根据接收到的路由键查找匹配的队列。

7.如果找到，将消息存入对应的队列，如果没有找到，会根据生产者的配置丢弃或者退回给生产者。

8.关闭信道。

9.管理连接。

* **消费者接收消息过程？**  

1.Producer先连接到Broker,建立连接Connection,开启一个信道(Channel)。

2.向Broker请求消费响应的队列中消息，可能会设置响应的回调函数。

3.等待Broker回应并投递相应队列中的消息，接收消息。

4.消费者确认收到的消息,ack。

5.RabbitMq从队列中删除已经确定的消息。

6.关闭信道。

7.关闭连接。

* **交换器无法根据自身类型和路由键找到符合条件队列时，有哪些处理？**  

mandatory ：true 返回消息给生产者。

mandatory: false 直接丢弃。

